<a href="https://colab.research.google.com/github/shawk17/563-shawk17/blob/main/genice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

On Google Colaboratory, you must install GenIce2 and extra plugins by yourself.


In [1]:
# install GenIce2 and extra plugins from PyPI via pip.
%pip install genice2 genice2-svg genice2-cage


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.7/304.7 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of genice2-svg to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.2/872.2 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 4.2 MB/s eta 0:00:00
  Created wheel for pairlist: filename=pairlist-0.6-cp311-cp311-manylinux_2_35_x86_64.whl size=31998 sha256=2b8c935fc21977c130e3dcddb5920904166df57c3b44bed0aeb4f7c5098f49f2
  Stored in directory: /root/.cache/pip/wheels/fe/f4/f6/bfb46169a454fde12dd069ce2a20c7bfdc04ad9a14a5eb38b2
Successfully built pairlist


## Output in Gromacs format.

A simplest example.

In [ ]:
from genice2.genice import GenIce
from genice2.plugin import Lattice, Format, Molecule

lattice = Lattice("ice11")
formatter = Format("gromacs")
water = Molecule("spce")
ice = GenIce(lattice).generate_ice(formatter, water=water)
print(ice)

## Generate an SVG image

Here we are using the Forms syntax of Google Colab to choose the ice type.

In [ ]:
from genice2.genice import GenIce
from genice2.plugin import Lattice, Format, Molecule

ice = "DDR"  # @param ["ice1h", "ice2", "ice3", "DDR"] {allow-input: true}

lattice = Lattice(ice)
formatter = Format("svg", shadow=True, rotate="y5,x5")
# formatter  = Format("twist", svg=True,rotatex=5,rotatey=5)
img = GenIce(lattice, reshape=[[2, 0, 0], [0, 2, 0], [0, 0, 2]]).generate_ice(formatter)

from IPython.display import SVG, display

display(SVG(img))

## Generate a PNG image

PNG renderer is also included in genice_svg package.

In [ ]:
from genice2.genice import GenIce
from genice2.plugin import Lattice, Format, Molecule

lattice = Lattice("CS2")
formatter = Format("png", shadow="#8881", encode=False, bg="#fff", rotate="x5,y5")
img = GenIce(lattice, reshape=[[2, 0, 0], [0, 2, 0], [0, 0, 2]]).generate_ice(formatter)

from IPython.display import display

display(img)

## Store the image in a file

Save it in a PNG file.

In [ ]:
from genice2.genice import GenIce
from genice2.plugin import Lattice, Format, Molecule

lattice = Lattice("ice3")
formatter = Format("png", shadow="#8881", encode=True, bg="#fff", rotate="x5,y5")
img = GenIce(lattice, reshape=[[2, 0, 0], [0, 2, 0], [0, 0, 2]]).generate_ice(formatter)

with open("ice3.png", "wb") as f:
    f.write(img)

## Analysis

Some statistical analyses are also available.

In [ ]:
from genice2.genice import GenIce
from genice2.plugin import Lattice, Format, Molecule
import numpy as np
from matplotlib import pyplot as plt
import random

lattice = Lattice("ice1c")
formatter = Format("_KG")
water = Molecule("spce")
for seed in range(1000, 1010):
    random.seed(seed)
    np.random.seed(seed)
    result = GenIce(lattice, reshape=[[10, 0, 0], [0, 10, 0], [0, 0, 10]]).generate_ice(
        formatter, water=water
    )

    kg = np.fromstring(result, dtype=float, sep=" ")
    N = kg.shape[0]
    kg = kg.reshape([N // 2, 2])
    plt.plot(kg[:, 0], kg[:, 1])

plt.xlabel("r / nm")
plt.ylabel("Kirkwood G")
plt.show()

## Logging

You may want to see the process log.

In [ ]:
from logging import getLogger, INFO, DEBUG, WARNING

logger = getLogger()
logger.setLevel(INFO)

from genice2.genice import GenIce
from genice2.plugin import Lattice, Format, Molecule

lattice = Lattice("ice1c")
formatter = Format("gromacs")
water = Molecule("spce")
ice = GenIce(
    lattice, reshape=[[16, 0, 0], [0, 16, 0], [0, 0, 16]], signature="Jupyter test"
).generate_ice(formatter, water=water)
with open("ice1c.gro", "w") as f:
    f.write(ice)

logger.setLevel(WARNING)

## Raw data

You can access the "raw" (internal) data.

In [ ]:
from genice2.genice import GenIce
from genice2.plugin import Lattice, Format, Molecule

lattice = Lattice("xFAU", rep=3)
formatter = Format("raw", stage=(1,))
raw = GenIce(lattice, signature="Aeroice 3xFAU").generate_ice(formatter)

raw

### Visualization by Plotly

Use the raw data to plot by yourself.

In [ ]:
waters = raw["reppositions"]
import plotly.graph_objects as go

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=waters[:, 0],
            y=waters[:, 1],
            z=waters[:, 2],
            marker=dict(size=10),
            mode="markers",
        )
    ]
)
fig.show()

## Integration with py3Dmol.

Py3Dmol works on Google Colaboratory.

In [ ]:
%pip install py3Dmol

In [ ]:
from genice2.genice import GenIce
from genice2.plugin import Lattice, Format, Molecule

lattice = Lattice("ice1h_unit")
formatter = Format("gromacs")
water = Molecule("spce")
ice = GenIce(lattice, reshape=[[4, 0, 0], [0, 4, 0], [0, 0, 4]]).generate_ice(
    formatter, water=water
)

In [ ]:
import py3Dmol

view = py3Dmol.view()
view.addModel(ice, "gro")
view.setStyle({"stick": {}})
view.addUnitCell()
view.zoomTo()

## Various file formats using MDAnalysis

In [ ]:
%pip install genice2-mdanalysis
exit(0)


In [ ]:
from genice2.genice import GenIce
from genice2.plugin import Lattice, Format, Molecule

lattice = Lattice("ice1h_unit")
formatter = Format("mdanalysis")
water = Molecule("spce")

# MDAnalysis universe
universe = GenIce(lattice, reshape=[[4, 0, 0], [0, 4, 0], [0, 0, 4]]).generate_ice(
    formatter, water=water
)

# Save as a .gro file.
allatoms = universe.select_atoms("all")
allatoms.write("1h_unit.pdb")